# 《统计方法在Meta分析研究可合并性中的应用》
## (Statistical Methods for Study Combinability in Meta-Analysis)

欢迎来到这门关于 Meta分析进阶统计方法的课程。今天我们将深入探讨在整合医学和生物学研究数据时，如何科学地判断研究的**可合并性 (Combinability)**，以及如何处理令人头疼的异质性和协变量不平衡问题。

## 第一部分：Meta分析基础入门 (Introduction & Basics)

### 1. 系统评价与 Meta 分析
- **系统评价 (Systematic Review)**：系统、全面地收集临床研究，采用严格评价标准筛选出符合质量标准的文献并定性或定量分析的过程。
- **Meta分析 (Meta-Analysis)**：系统评价中常用的定量统计学方法，合并效应量。

### 2. Meta分析的 7 个步骤
1. 提出问题 (Formulate the problem)
2. 文献检索 (Literature search)
3. 文献筛选 (Screening abstracts and full-text)
4. 数据提取 (Data extraction)
5. 质量评估 (Quality assessment / Risk of bias)
6. 数据综合与分析 (Data synthesis and analysis)
7. 结果解释与报告 (Interpretation and reporting)

### 3. 理解偏差 (Bias) 的隐性威胁
- 发表偏倚 (Publication Bias)
- 赞助偏倚 (Sponsorship Bias)
- 选择偏倚 (Selection Bias)

## 第二部分：异质性与统计模型 (Heterogeneity & Models)

**异质性 (Heterogeneity)** 指的是纳入各项研究之间在临床、方法学及统计学的差异。
- **固定效应模型 (Fixed Effect Model)**：假设均拥有真实的治疗效果。适用于异质性小。
- **随机效应模型 (Random Effects Model)**：包含抽样误差和研究间变异。适用于异质性明显。

## 第三部分：协变量不平衡与 Meta 回归 (Covariate Imbalance)

### 可合并性的两个层级
- **基本可合并性 (Basic combinability)**：不对协变量调整前，受试者特征是否足够相似。
- **边际可合并性 (Marginal combinability)**：对重要混杂因素调整后。

### 控制混杂的常用方法
- 倾向评分匹配法 (Propensity Score Method)
- **经验累积分布函数法 (ECDF Method)**: 本课重点！


## 第四部分：Python 代码实战 (Data Practice)

我们将使用真实的数据集 `col1.csv`。为了实现对某些协变量（如糖尿病患病率 `pdiab`、男性比例 `pmale`，平均年龄 `age`）的基线平衡性检验，我们需要一个特殊的技巧。

由于 `col1.csv` 提供的是各个试验的**汇总层级数据 (Summary level/Study level data)**，比如均值和标准差，我们需要先通过 Monte Carlo 模拟，将其“平展（Explode）”或“还原”成**个体患者数据 (Individual Patient Data, IPD)**的伪样本。

这一步在原文中是通过 `rnorm` 等分布函数给每个试验的每个患者生成带有小扰动（Perturbation）的假定实际值。

接下来，我们将使用 **Anderson-Darling k-sample test** 来检验实验组(exp)和对照组(ctrl)间的经验分布是否存在差异。如果差异过大（P < 0.05），说明存在协变量不平衡，我们需要使用 Leave-One-Out (逐个剔除) 的算法，寻找“导致不平衡罪魁祸首”的那个 Study 并剔除它，直到达到平衡。


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('col1.csv', sep=';')  
df_focus = df[df['nation'] == 'USA'].copy()

print(f"USA 区实验数: {len(df_focus['study'].unique())}")
df_focus.head()

### 算法核心：Leave-One-Out (逐一剔除平衡算法)

以下是将 R 语言代码转写为 Python 的核心 `balance` 算法。该算法能够帮我们在汇总数据中判断不同试验间的基线是否处于“基本可合并”状态。

In [ ]:
def ad_test_k_sample(sample1, sample2):
    '''
    封装 scipy.stats.anderson_ksamp
    返回检验统计量和对应的 p-value
    '''
    # 需要移除 NaN 并且必须转换为列表才能过 anderson_ksamp
    s1 = sample1[~np.isnan(sample1)]
    s2 = sample2[~np.isnan(sample2)]
    
    if len(s1) < 2 or len(s2) < 2:
        return np.inf, 0.0 # 当没有样本时，返回极大值和极小p值以防止毁坏组别结构
    
    try:
        # 抛出UserWarning如果p值大于0.25
        res = stats.anderson_ksamp([s1, s2])
        return res.statistic, res.pvalue
    except OverflowError:
        return np.inf, 0.0
    except Exception as e:
        warnings.warn(f"AD Test Warning: {str(e)}")
        return np.inf, 0.0

def balance(data, variable, group_col, digits=8):
    '''
    留一法（Leave-One-Out）评估哪个 study 会严重影响平衡性。
    对应 R 中的 balance 函数。
    '''
    unique_studies = data['study'].unique()
    results = []
    
    # 获取属于组1和组2的完整数据框
    groups = data[group_col].unique()
    if len(groups) < 2:
        return pd.DataFrame() # 组别不足
        
    group1_val = groups[0]
    group2_val = groups[1]
    
    sa1 = data[data[group_col] == group1_val]
    sa2 = data[data[group_col] == group2_val]
    
    for i in unique_studies:
        # 当剔除 study 'i' 时的剩余数据
        a1 = sa1.loc[sa1['study'] != i, variable].round(digits).values
        a2 = sa2.loc[sa2['study'] != i, variable].round(digits).values
        
        stat, pval = ad_test_k_sample(a1, a2)
        results.append({
            'Study_Omitted': i,
            'AD_Stat': stat,
            'P_Value': pval
        })
        
    return pd.DataFrame(results).set_index('Study_Omitted')

### 案例 1: 年龄分布 (Age) 的平衡与 ECDF 绘图

我们首先看 `age`。

In [ ]:
# IPD级别患者数据，不需要进行 Explode 操作，直接使用即可提取
da = df_focus[['study', 'age', 'gr', 'sda', 'pts']].copy()

# 引入微小均匀随机扰动，防止 ties 影响 AD Test，且不会改变原分布宏观结构
np.random.seed(42)
da['age_pert'] = da['age'] + np.random.uniform(-1e-6, 1e-6, size=len(da))

# AD Test over all trials
a1_all = da.loc[da['gr'] == 'ctrl', 'age_pert'].round(8).values
a2_all = da.loc[da['gr'] == 'exp', 'age_pert'].round(8).values

print("=== 整体伪样本的 Anderson-Darling Test ===")
global_stat, global_pval = ad_test_k_sample(a1_all, a2_all)
print(f"AD Statistic: {global_stat:.4f}, P-Value: {global_pval:.4f}")

# 画 ECDF
def plot_ecdf(s1, s2, label1='Ctrl', label2='Exp', title='Original Data'):
    plt.figure(figsize=(8,5))
    sns.ecdfplot(s1, label=label1, color='red', drawstyle='steps-post')
    sns.ecdfplot(s2, label=label2, color='blue', linestyle='--', drawstyle='steps-post')
    plt.title(title)
    plt.legend()
    plt.show()

plot_ecdf(a1_all, a2_all, title="ECDF for Age - All Studies")

In [ ]:
# 留一法循环剔除
print("=== 开始 Leave-One-Out 剔除 ===")
dat = da.copy()

unique_studies = dat['study'].unique()
nstud = len(unique_studies)
deleted_trials = []

import warnings

# 如果研究过少，无法进行留一探讨
if nstud < 3:
    print(f"警告: 本区域数据只包含 {nstud} 个研究: {list(unique_studies)}。")
    print("数据不足以进行有科学价值的留一法比较 (至少需包含3个研究以上)。此处展示逻辑将跳过剔除。")
else:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for j in range(1, nstud + 1):
            ba = balance(dat, variable="age_pert", group_col="gr", digits=8)
            
            if ba.empty:
                print("组别不足或无法计算，终止。")
                break
                
            # 找出剔除后AD统计量最小（也就是剩余样本最平衡）的那个 Study
            minimum = ba['AD_Stat'].idxmin()
            # 最小 AD 对应的 p 值
            min_pval = ba.loc[minimum, 'P_Value']
            
            # 从总集中正式剔除这个 study
            dat = dat[dat['study'] != minimum]
            deleted_trials.append(minimum)
            print(f"迭代 {j}: 剔除 Trial {minimum}, 此时 p-value:{min_pval:.4f}")
            
            a1_curr = dat.loc[dat['gr'] == 'ctrl', 'age_pert'].values
            a2_curr = dat.loc[dat['gr'] == 'exp', 'age_pert'].values
            
            # 如果不显著（p > 0.06），说明两组分布已经没有显著差异，可以停止剔除
            if min_pval > 0.06:
                print(f"==> 达到平衡，终止！已剔除研究: {deleted_trials}")
                plot_ecdf(a1_curr, a2_curr, title=f"ECDF for Age - Balanced (Iteration {j})")
                break
            
            if len(dat['study'].unique()) < 2:
                print("==> 剩余研究不足 2 个，无法继续留一比较，触发安全防线。")
                break


### 案例 2: 年龄分布的边际可合并性问题 (Marginal Combinability)

有时候因为地域或政策差异，不同大类（比如欧洲 EU 和 非欧洲 OT）的基线存在极大鸿沟。我们绘制 ECDF，寻找 Common Support。

In [ ]:
# 比较 USA 与 China 的 Age
# 注意: 这里用 col1.csv 的全部数据
df_usa_age = df[df['nation'] == 'USA'].copy()
df_chn_age = df[df['nation'] == 'China'].copy()

# IPD级别患者数据，不用重复展开，直接获取真实特征分布 (修复错误)
b5 = df_usa_age['age'].values
b6 = df_chn_age['age'].values

plt.figure(figsize=(10, 5))
sns.ecdfplot(b5, color='red', label='USA', linewidth=2)
sns.ecdfplot(b6, color='blue', label='China', linestyle='--', linewidth=2)
plt.title('Patient-Level Variables: Age ECDFs (USA vs China)')
plt.xlabel('Mean Age')
plt.ylabel('ECDF')
plt.legend()
plt.show()

print("我们可以从上面两组 ECDF 看到明显的右移效应。两者分布存在明显缝隙。")

---
## 第五部分：Monte Carlo "展开" 技巧 (Explode from Summary to IPD)

R 原始代码中，数据集 `col1` 是**汇总层级（study-level）**的：每行代表一项试验，记录了各组的均値、标准差和样本量。

为了对个体患者特征（如 `pdiab`、`pmale`）进行分布检验，需要先将其展开为**个体患者数据（IPD）**的伪样本。

```r
# R 语言版本
da <- as.data.frame(lapply(ro, function(x) rep(x, A1$pts)))  # 按 pts 重复
da$pdiab <- rnorm(length(da$pdiab), da$pdiab, da$sdd)        # 加正态扰动
```

Python 等价实现：
- 用 `df.iterrows()` + 循环 `repeat` 按每个 study 的患者数重复行
- 用 `np.random.normal(mean, sd)` 为每位“虚拟患者”加入微小正态扰动

**为什么要加扰动（Perturbation）？** 原始汇总数据中每个试验内所有患者的特征値相同（即 ties），AD Test 对 ties 非常敏感。加入微小随机扰动可以打破 ties，同时保持宏观分布结构不变。


### 案例 3：糖尿病患病率 `pdiab` 的基本可合并性 (Basic Combinability)

对应 R 代码中的 `DIAB` 部分：
1. 从 `df_focus` 中提取 `pdiab`、`gr`、`sdd`、`pts` 列
2. 按 `pts` 展开为 IPD，并加正态扰动
3. 对全组进行 AD Test，绘制 ECDF
4. 运行 Leave-One-Out 循环直到 p > 0.06


In [ ]:
np.random.seed(42)

ro_diab = df_focus[['study', 'pdiab', 'gr', 'sdd', 'pts']].copy()

rows_diab = []
for _, row in ro_diab.iterrows():
    n = int(row['pts'])
    for _ in range(n):
        rows_diab.append(row.to_dict())
da_diab = pd.DataFrame(rows_diab).reset_index(drop=True)
da_diab['pdiab'] = np.random.normal(da_diab['pdiab'].values, da_diab['sdd'].values)

a1_diab_all = da_diab.loc[da_diab['gr'] == 'ctrl', 'pdiab'].round(8).values
a2_diab_all = da_diab.loc[da_diab['gr'] == 'exp',  'pdiab'].round(8).values

print('=== pdiab Anderson-Darling Test (all studies) ===')
gstat, gpval = ad_test_k_sample(a1_diab_all, a2_diab_all)
print(f'AD Statistic: {gstat:.4f},  P-Value: {gpval:.4f}')

plot_ecdf(a1_diab_all, a2_diab_all, label1='Ctrl', label2='Exp',
          title='ECDF for pdiab - All Studies (Original)')


In [ ]:
print('=== pdiab Leave-One-Out ===')
dat_diab = da_diab.copy()
deleted_diab = []
nstud_diab = len(dat_diab['study'].unique())

if nstud_diab < 3:
    print(f'Warning: only {nstud_diab} studies, skipping LOO.')
else:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for j in range(1, nstud_diab + 1):
            ba = balance(dat_diab, variable='pdiab', group_col='gr', digits=8)
            if ba.empty:
                print('Not enough groups, stopping.')
                break
            minimum = ba['AD_Stat'].idxmin()
            min_pval = ba.loc[minimum, 'P_Value']
            dat_diab = dat_diab[dat_diab['study'] != minimum]
            deleted_diab.append(minimum)
            print(f'Iter {j}: removed Trial {minimum}, p-value={min_pval:.4f}')
            if min_pval > 0.06:
                a1_b = dat_diab.loc[dat_diab['gr'] == 'ctrl', 'pdiab'].values
                a2_b = dat_diab.loc[dat_diab['gr'] == 'exp',  'pdiab'].values
                print(f'==> Balance reached! Removed studies: {deleted_diab}')
                plot_ecdf(a1_b, a2_b, title=f'ECDF for pdiab - Balanced (Iter {j})')
                break
            if len(dat_diab['study'].unique()) < 2:
                print('==> Fewer than 2 studies remain, stopping.')
                break


### 案例 4：男性比例 `pmale` 的基本可合并性 (Basic Combinability)

对应 R 代码中的 `MALE` 部分。流程与 `pdiab` 完全相同，需注意 R 中使用的标准差列为 `sdm`。


In [ ]:
np.random.seed(42)

ro_male = df_focus[['study', 'pmale', 'gr', 'sdm', 'pts']].copy()

rows_male = []
for _, row in ro_male.iterrows():
    n = int(row['pts'])
    for _ in range(n):
        rows_male.append(row.to_dict())
da_male = pd.DataFrame(rows_male).reset_index(drop=True)
da_male['pmale'] = np.random.normal(da_male['pmale'].values, da_male['sdm'].values)

a1_male_all = da_male.loc[da_male['gr'] == 'ctrl', 'pmale'].round(8).values
a2_male_all = da_male.loc[da_male['gr'] == 'exp',  'pmale'].round(8).values

print('=== pmale Anderson-Darling Test (all studies) ===')
gstat_m, gpval_m = ad_test_k_sample(a1_male_all, a2_male_all)
print(f'AD Statistic: {gstat_m:.4f},  P-Value: {gpval_m:.4f}')

plot_ecdf(a1_male_all, a2_male_all, label1='Ctrl', label2='Exp',
          title='ECDF for pmale - All Studies (Original)')

print('\n=== pmale Leave-One-Out ===')
dat_male = da_male.copy()
deleted_male = []
nstud_male = len(dat_male['study'].unique())

if nstud_male < 3:
    print(f'Warning: only {nstud_male} studies, skipping LOO.')
else:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        for j in range(1, nstud_male + 1):
            ba = balance(dat_male, variable='pmale', group_col='gr', digits=8)
            if ba.empty:
                print('Not enough groups, stopping.')
                break
            minimum = ba['AD_Stat'].idxmin()
            min_pval = ba.loc[minimum, 'P_Value']
            dat_male = dat_male[dat_male['study'] != minimum]
            deleted_male.append(minimum)
            print(f'Iter {j}: removed Trial {minimum}, p-value={min_pval:.4f}')
            if min_pval > 0.06:
                a1_b = dat_male.loc[dat_male['gr'] == 'ctrl', 'pmale'].values
                a2_b = dat_male.loc[dat_male['gr'] == 'exp',  'pmale'].values
                print(f'==> Balance reached! Removed studies: {deleted_male}')
                plot_ecdf(a1_b, a2_b, title=f'ECDF for pmale - Balanced (Iter {j})')
                break
            if len(dat_male['study'].unique()) < 2:
                print('==> Fewer than 2 studies remain, stopping.')
                break


---
### 案例 5：国家层级边际可合并性 (Marginal Combinability by Nation)

R 代码最后一段展示了**边际可合并性**的另一维度：不同国家/地区的研究是否可以合并？

以 **EU（欧洲）vs OT（非欧洲）** 的年龄分布为例，R 中采用了两种 ECDF 可视化方式：

| 方法 | R 实现 | Python 等价 |
|---|---|---|
| **阶梯 ECDF（Step）** | `rep(age, pts)` + `ecdf()` | `np.repeat(age, pts)` + step plot |
| **平滑 ECDF（Smooth）** | `rnorm(n, age, sda)` + `ecdf()` | `np.random.normal(age, sda)` + `sns.ecdfplot` |

下面将这两种方法并列展示，对比“离散化”观点与“连续化”观点下各地区年龄分布差异。


In [ ]:
np.random.seed(42)

nations = df['nation'].unique()
if 'eu' in nations:
    df_eu = df[df['nation'] == 'eu'].copy()
    df_ot = df[df['nation'] == 'ot'].copy()
    label_eu, label_ot = 'EU', 'Non-EU'
else:
    df_eu = df[df['nation'] == 'USA'].copy()
    df_ot = df[df['nation'] == 'China'].copy()
    label_eu, label_ot = 'USA', 'China'

b5 = np.repeat(df_eu['age'].values, df_eu['pts'].values.astype(int))
b6 = np.repeat(df_ot['age'].values, df_ot['pts'].values.astype(int))

rows_eu = []
for _, row in df_eu[['study', 'age', 'sda', 'pts']].iterrows():
    rows_eu.extend([{'age': row['age'], 'sda': row['sda']}] * int(row['pts']))
na_eu_df = pd.DataFrame(rows_eu)
na1 = np.random.normal(na_eu_df['age'].values, na_eu_df['sda'].values).round(8)

rows_ot = []
for _, row in df_ot[['study', 'age', 'sda', 'pts']].iterrows():
    rows_ot.extend([{'age': row['age'], 'sda': row['sda']}] * int(row['pts']))
na_ot_df = pd.DataFrame(rows_ot)
na2 = np.random.normal(na_ot_df['age'].values, na_ot_df['sda'].values).round(8)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].step(sorted(b5), [i/len(b5) for i in range(1, len(b5)+1)],
             color='red', label=label_eu, linewidth=1.5)
axes[0].step(sorted(b6), [i/len(b6) for i in range(1, len(b6)+1)],
             color='blue', linestyle='--', label=label_ot, linewidth=1.5)
axes[0].set_title(f'Step ECDF: Age by Nation ({label_eu} vs {label_ot})')
axes[0].set_xlabel('Mean Age')
axes[0].set_ylabel('ECDF')
axes[0].legend()

sns.ecdfplot(na1, ax=axes[1], color='red', label=label_eu, drawstyle='steps-post')
sns.ecdfplot(na2, ax=axes[1], color='blue', linestyle='--', label=label_ot, drawstyle='steps-post')
axes[1].set_title(f'Smooth ECDF: Age by Nation ({label_eu} vs {label_ot})')
axes[1].set_xlabel('Mean Age')
axes[1].set_ylabel('ECDF')
axes[1].legend()

plt.tight_layout()
plt.show()

print('If the two ECDF curves show a clear gap, the two regions lack common support')
print('in the age distribution -- merging them directly raises marginal combinability concerns.')
